In [1]:
import os
import numpy as np
import pandas as pd
import eipy.ei as e
import tensorflow as tf
from importlib import reload
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import pandas as pd
from eipy.additional_ensembles import MeanAggregation, CES
from eipy.metrics import fmax_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Concatenate, LSTM

2024-02-01 20:07:41.143242: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 20:07:41.170528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 20:07:41.170555: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 20:07:41.171309: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 20:07:41.175906: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 20:07:41.176372: I tensorflow/core/platform/cpu_feature_guard.cc:1

data

In [2]:
directory = "/home/opc/eipy/eipy/COVID-19 data"
data={}
for file_name in os.listdir(directory):
    if file_name.endswith('.csv'):
        key = file_name.split('.')[0]
        file_path = os.path.join(directory, file_name)
        df = pd.read_csv(file_path)
        df = df.drop(columns=["NEW_MASKED_MRN"])
        imputer = KNNImputer(n_neighbors=5)
        df = imputer.fit_transform(df)
        data[key] = df

In [3]:
labels = pd.read_csv('/home/opc/eipy/eipy/COVID-19 data/Basics/outcome.csv')
labels = labels["DECEASED_INDICATOR"]

In [4]:
X_train = {}
X_test = {}
for k, v in data.items():
    X_train[k], X_test[k], y_train, y_test = train_test_split(v, labels, test_size=0.2, stratify=labels)

In [5]:
for k,v in X_train.items():
    print(k, v.shape)

admission (3826, 23)
comorbidities (3826, 22)
labs (3826, 44)
vitals (3826, 9)
xgboost (3826, 98)


dl bps

In [6]:
input_dim = 98

# Architecture 1: Simple Feedforward Neural Network
model_1 = Sequential()
model_1.add(Dense(64, input_dim=input_dim, activation='relu'))
model_1.add(Dense(32, activation='relu'))
model_1.add(Dense(1, activation='sigmoid'))
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Architecture 2: Deep Feedforward Neural Network with Dropout
model_2 = Sequential()
model_2.add(Dense(128, input_dim=input_dim, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dropout(0.3))
model_2.add(Dense(32, activation='relu'))
model_2.add(Dense(1, activation='sigmoid'))
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Architecture 3: Wide and Deep Neural Network
input_layer = Input(shape=(input_dim,))
wide_branch = Dense(32, activation='relu')(input_layer)
deep_branch = Dense(64, activation='relu')(input_layer)
deep_branch = Dense(32, activation='relu')(deep_branch)
merged = Concatenate()([wide_branch, deep_branch])
output_layer = Dense(1, activation='sigmoid')(merged)
model_3 = Model(inputs=input_layer, outputs=output_layer)
model_3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Architecture 4: Deep Neural Network without Sequential Structure
model_4 = Sequential()
model_4.add(Dense(128, input_dim=input_dim, activation='relu'))
model_4.add(Dense(64, activation='relu'))
model_4.add(Dense(32, activation='relu'))
model_4.add(Dense(1, activation='sigmoid'))
model_4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

d_base_predictors = {"simple" : model_1,
                     "deep_w_dropout" : model_2,
                     "wide_n_deep" : model_3,
                     "deeper" : model_4}

In [7]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True),
}

In [8]:
ensemble_predictors = {
                    'Mean' : MeanAggregation(),
                    'CES' : CES(scoring=lambda y_test, y_pred: fmax_score(y_test, y_pred)[0]),
                    'S.ADAB': AdaBoostClassifier(),
                    'S.XGB': XGBClassifier(),
                    'S.DT': DecisionTreeClassifier(),
                    "S.RF": RandomForestClassifier(),
                    'S.GB': GradientBoostingClassifier(),
                    'S.KNN': KNeighborsClassifier(),
                    'S.LR': LogisticRegression(),
                    'S.NB': GaussianNB(),
                    'S.MLP': MLPClassifier(),
                    'S.SVM': SVC(probability=True),
}

In [9]:
metrics = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

EI

In [10]:
####HI OVER HERE###
reload(e)
###HEYOOOOOO#

<module 'eipy.ei' from '/home/opc/eipy/eipy/ei.py'>

In [11]:
EI = e.EnsembleIntegration(
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy="undersampling",
                        sampling_aggregation=None,
                        n_jobs=-1,
                        metrics=metrics,
                        random_state=38,
                        project_name="diabetes",
                        model_building=True,
                        )

In [12]:
for k,v in X_train.items():
    if k == "xgboost":
        EI.fit_base(v, y_train, modality_name=k, base_predictors=d_base_predictors)
    else:
        EI.fit_base(v, y_train, modality_name=k, base_predictors=base_predictors)

Training base predictors on admission...
        
... for ensemble performance analysis...


Generating ensemble training data: |          |  0%2024-02-01 20:07:48.029036: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 20:07:48.029034: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 20:07:48.029034: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 20:07:48.029034: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 20:07:48.048163: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 20:07:48.048164: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 20:07:48.055700: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could no


... for final ensemble...


Generating ensemble training data: |██████████|100%
Training final base predictors: |██████████|100%




Training base predictors on comorbidities...
        
... for ensemble performance analysis...


...

Generating final ensemble training data: |██████    | 60%

96/96 [==============================] - 0s 656us/step


Generating final ensemble training data: |████████  | 80%

96/96 [==============================] - 0s 649us/step


Generating final ensemble training data: |██████████|100%

In [13]:
EI.base_summary["metrics"]

modality       admission                                                    \
base predictor      ADAB        DT        GB       KNN        LR       MLP   
f_max           0.433340  0.433251  0.433340  0.433251  0.433518  0.433251   
auc             0.492454  0.502848  0.506390  0.501474  0.505765  0.503719   
mcc            -0.008758  0.005096  0.007398 -0.006016  0.004593  0.004975   

modality                                                ...    vitals  \
base predictor        NB        RF       SVM       XGB  ...        LR   
f_max           0.433978  0.433457  0.433251  0.433251  ...  0.435808   
auc             0.497834  0.494016  0.482706  0.494192  ...  0.514060   
mcc            -0.000469 -0.001810 -0.026807 -0.001469  ...  0.009723   

modality                                                          \
base predictor       MLP        NB        RF       SVM       XGB   
f_max           0.434156  0.434598  0.434711  0.433251  0.434656   
auc             0.505766  0.511719  0.512455  0.491140  0.509743   
mcc             0.015094  0.016957  0.028418 -0.013803  0.002484   

modality              xgboost                                  
base predictor deep_w_dropout    deeper    simple wide_n_deep  
f_max                0.433251  0.473534  0.496547    0.495100  
auc                  0.524233  0.630330  0.663893    0.677873  
mcc                  0.044455  0.177974  0.238724    0.246462  

[3 rows x 44 columns]

In [14]:
EI.fit_ensemble(ensemble_predictors=ensemble_predictors)

Analyzing ensembles: |██████████|100%
Training final ensemble models: |██████████|100%


In [15]:
EI.ensemble_summary["metrics"]

,Mean,CES,S.ADAB,S.XGB,S.DT,S.RF,S.GB,S.KNN,S.LR,S.NB,S.MLP,S.SVM
f_max,0.450209,0.515334,0.51094,0.517722,0.433251,0.520885,0.529830,0.469097,0.522593,0.499697,0.499196,0.486643
auc,0.602412,0.690091,0.68302,0.697476,0.570976,0.710068,0.722275,0.627240,0.713550,0.679760,0.680925,0.656368
mcc,0.132644,0.255541,0.22092,0.190408,0.137126,0.175244,0.226469,0.150354,0.204881,0.231117,0.187687,0.121121


In [16]:
preferred_ensemble_key = EI.ensemble_summary["metrics"].loc["auc"].idxmax()
print(preferred_ensemble_key)
y_pred = EI.predict(X_dict=X_test, ensemble_model_key=preferred_ensemble_key)
roc_auc_score(y_test, y_pred)

S.GB
30/30 [==============================] - 0s 742us/step


0.7360071981677391

In [17]:
from eipy.interpretation import PermutationInterpreter

interpreter = PermutationInterpreter(EI=EI,
                                     metric=lambda y_test, y_pred: roc_auc_score(y_test, y_pred),
                                     ensemble_predictor_keys=preferred_ensemble_key,
                                     n_jobs=-1)

In [19]:
EI.ensemble_predictors.keys()

dict_keys(['Mean', 'CES', 'S.ADAB', 'S.XGB', 'S.DT', 'S.RF', 'S.GB', 'S.KNN', 'S.LR', 'S.NB', 'S.MLP', 'S.SVM'])

In [18]:
interpreter.rank_product_score(X_dict=X_test, y=y_test)

Interpreting ensembles...



Calculating local feature ranks: |████████  | 80%

30/30 [==============================] - 0s 794us/step


Calculating local feature ranks: |██████████|100%

30/30 [==============================] - 0s 815us/step


KeyError: 'S'